In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from fractions import Fraction
from math import gcd

def plot_multiple_phasor_sum(phasors, input_thickness=0.005, resultant_thickness=0.007):
    """
    Plot multiple phasors and their successive sums with customizable thickness.
    
    Parameters:
    phasors (list): List of tuples, each containing (magnitude, angle) in degrees
    input_thickness (float): Thickness of input phasor arrows (default: 0.005)
    resultant_thickness (float): Thickness of resultant phasor arrow (default: 0.007)

    phasors example
    phasors = [
        (1.0, 90),   # Phasor U1
        (1.0, 90-1*30),  # Phasor U2
        (-1.0, 90-1*180), # Phasor -U7
        (-1.0, 90-1*210)   # Phasor -U8
    ]

    Notes:
    Uses quiver() to plot arrows representing:
       First phasor (blue)
       Second phasor (red, starting from the tip of the first)
       Resultant phasor (green)

    The "angles='xy'" parameter ensures the arrows point in the correct direction based on the calculated components

    """
    # Initialize lists to store components
    x_components = []
    y_components = []
    
    # Calculate components for each phasor
    for magnitude, angle in phasors:
        theta = np.radians(angle)
        x = magnitude * np.cos(theta)
        y = magnitude * np.sin(theta)
        x_components.append(x)
        y_components.append(y)

    # Create the plot
    plt.figure(figsize=(8, 6))
    ax = plt.gca()
    
    # Starting point
    current_x = 0
    current_y = 0
    
    # Plot each phasor
    cycle_colors = ['black', 'grey']
    num_colors = len(cycle_colors)
    for i, ((magnitude, angle)) in enumerate(phasors):
        color_to_use = cycle_colors[i % num_colors]
        x = x_components[i]
        y = y_components[i]
        ax.quiver(current_x, current_y, x, y, angles='xy', scale_units='xy', scale=1, 
                 color=color_to_use, width=input_thickness,
                 label=f'Phasor {i+1}: {magnitude}∠{angle}°')
        current_x += x
        current_y += y
    
    # Plot resultant
    resultant_x = sum(x_components)
    resultant_y = sum(y_components)
    magnitude_resultant = np.sqrt(resultant_x**2 + resultant_y**2)
    angle_resultant = np.degrees(np.arctan2(resultant_y, resultant_x))
    ax.quiver(0, 0, resultant_x, resultant_y, angles='xy', scale_units='xy', scale=1, 
             color='green', width=resultant_thickness,
             label=f'Geometric sum: {magnitude_resultant:.2f}∠{angle_resultant:.1f}°')

    # Set plot parameters
    ax.set_aspect('equal')
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)

    # Set limits
    max_range = max(magnitude_resultant, *[mag for mag, _ in phasors]) * 2
    plt.xlim(-max_range, max_range)
    plt.ylim(-max_range, max_range)

    # Add labels and title
    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.title('Multiple Phasor Addition')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # Show the plot
    plt.show()

# Example usage:
'''The if __name__ == "__main__": section shows an example usage when running the script directly, 
but you can call the function from anywhere with any valid values. 
The docstring explains what the function does and what parameters it expects.'''

def calculate_alpha_u(Q, p, m, y):
    """
    Calculate alpha_u based on motor parameters.
    
    Parameters:
    Q (int): Number of slots
    p (int): Number of poles
    m (int): Number of phases
    y (int): Winding pitch
    
    Returns:
    float: alpha_u in electrical degrees
    """
    # Calculate basic parameters
    PP = int(p/2)  # Number of pole-pairs
    t = gcd(Q, PP)  # Largest common divider between Q and PP
    
    # Calculate q (slots per pole per phase)
    q = Fraction(Q, p*m)
    z = q.numerator
    n = q.denominator
    
    # Calculate alpha_u
    if t == PP:
        alpha_u = 360 * PP / Q
        alpha_z = alpha_u
    else:
        Qp = Q/t
        alpha_z = 360/Qp
        if n%2 != 0:  # If n is odd => First grade winding
            alpha_u = n * alpha_z
        else:
            alpha_u = n/2 * alpha_z
            
    return alpha_u


def plot_phasor_diagram(Q, p, m, alpha_u):
    """
    Generate and plot a phasor diagram for a motor based on input parameters and alpha_u.
    
    Parameters:
    Q (int): Number of slots
    p (int): Number of poles
    m (int): Number of phases
    alpha_u (float): Displacement angle between consecutive phasors in degrees
    
    Returns:
    None: Displays the phasor diagram
    """
    # Calculate basic parameters
    PP = int(p/2)  # Number of pole-pairs
    t = gcd(Q, PP)  # Largest common divider between Q and PP
    Qp = Q/t  # Total number of phasors
    phasors_per_layer = int(Q // t)  # Phasors per layer
    
    # Initialize plot
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)
    
    # Define radii for layers
    radii = [1 + 0.5 * i for i in range(t)]
    
    # Starting angle (90 degrees for positive y-axis)
    current_angle = 90
    
    # Arrowhead length
    head_length = 0.05
    
    # Define phasors to be colored red
    U_phasors = [1, 8, 7, 2]
    
    # Plot phasors
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            angle_rad = np.deg2rad(current_angle)
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Set color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset/180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset/180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            phasor_number += 1
            current_angle -= alpha_u
    
    # Draw circles for layers if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)
    
    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)
    
    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')
    
    # Save the plot instead of showing it
    plt.savefig('phasor_diagram.png')


def generate_phase_u(Q, p, m, alpha_u):
    """
    Generate the phase U array for a motor based on input parameters.
    
    Parameters:
    Q (int): Number of slots
    p (int): Number of poles
    m (int): Number of phases
    alpha_u (float): Displacement angle between consecutive phasors in degrees
    
    Returns:
    list: Array of phasor numbers for phase U
    """
    # Calculate basic parameters
    PP = p // 2  # Number of pole-pairs
    t = gcd(Q, PP)  # Largest common divider between Q and PP
    phasors_per_phase = Q // t // m  # Number of phasors per phase (length of phase U array)
    
    if phasors_per_phase < 4:
        raise ValueError("Phase U array requires at least 4 phasors (Q/t/m >= 4)")
    
    # Calculate angles for all phasors
    angles = []
    for k in range(1, Q + 1):
        # Angle for phasor k: 90 - (k-1) * alpha_u, normalized to [0, 360)
        angle = (90 - (k - 1) * alpha_u) % 360
        angles.append((angle, k))  # Store (angle, phasor_number)
    
    # Sort angles in ascending order (counterclockwise progression)
    angles.sort()
    
    # Find phasor 1's position
    phasor_1_angle = None
    phasor_1_index = None
    for i, (angle, phasor) in enumerate(angles):
        if phasor == 1:
            phasor_1_angle = angle
            phasor_1_index = i
            break
    
    # Initialize phase U array
    phase_u = [1]  # First value is phasor 1
    
    # Second value: phasor immediately counterclockwise from phasor 1
    next_index = (phasor_1_index + 1) % Q
    phase_u.append(angles[next_index][1])
    
    # Third value: Q/t/m + 1 phasors away from second value counterclockwise
    steps = phasors_per_phase + 1
    third_index = (next_index + steps) % Q
    phase_u.append(angles[third_index][1])
    
    # Fourth value: phasor immediately counterclockwise from third value
    fourth_index = (third_index + 1) % Q
    phase_u.append(angles[fourth_index][1])
    
    return phase_u

if __name__ == "__main__":

    Q = 21 # Q (int): Number of slots
    p = 22 # p (int): Number of poles # "p" is the no. of poles NOTE: Pyrhonen use "p" as Pole-pair
    m = 3 # m (int): Number of phases
    y = 1 # y (int): Winding pitch/ coil throw
    # Define phasors as (magnitude, angle) pairs
    
    # Example 2.23
    # alpha_u calculation requires Q, p, m, and y
    alpha_u = calculate_alpha_u(Q, p, m, y)

    phase_u_array = generate_phase_u(Q, p, m, alpha_u)
    print(phase_u_array)

    alpha_u1 = 90
    phasor_list = [
        (1.0, alpha_u1%360), # Phasor U1
        (1.0, (alpha_u1-7*alpha_u)%360), # Phasor U8
        (-1.0, (alpha_u1-6*alpha_u)%360), # Phasor -U7
        (-1.0, (alpha_u1-1*alpha_u)%360) # Phasor -U2
    ]
    # phasor diagram plot requires Q, p, m, and alpha_u
    plot_phasor_diagram(Q, p, m, alpha_u)
    # Call with custom thicknesses
    plot_multiple_phasor_sum(phasor_list, input_thickness=0.004, resultant_thickness=0.006)

In [ ]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from fractions import Fraction
from math import gcd

def plot_multiple_phasor_sum(phasors, input_thickness=0.005, resultant_thickness=0.007):
    """
    Plot multiple phasors and their successive sums with customizable thickness.
    
    Parameters:
    phasors (list): List of tuples, each containing (magnitude, angle) in degrees
    input_thickness (float): Thickness of input phasor arrows (default: 0.005)
    resultant_thickness (float): Thickness of resultant phasor arrow (default: 0.007)

    phasors example
    phasors = [
        (1.0, 90),   # Phasor U1
        (1.0, 90-1*30),  # Phasor U2
        (-1.0, 90-1*180), # Phasor -U7
        (-1.0, 90-1*210)   # Phasor -U8
    ]

    Notes:
    Uses quiver() to plot arrows representing:
       First phasor (blue)
       Second phasor (red, starting from the tip of the first)
       Resultant phasor (green)

    The "angles='xy'" parameter ensures the arrows point in the correct direction based on the calculated components

    """
    # Initialize lists to store components
    x_components = []
    y_components = []
    
    # Calculate components for each phasor
    for magnitude, angle in phasors:
        theta = np.radians(angle)
        x = magnitude * np.cos(theta)
        y = magnitude * np.sin(theta)
        x_components.append(x)
        y_components.append(y)

    # Create the plot
    plt.figure(figsize=(8, 6))
    ax = plt.gca()
    
    # Starting point
    current_x = 0
    current_y = 0
    
    # Plot each phasor
    cycle_colors = ['black', 'grey']
    num_colors = len(cycle_colors)
    for i, ((magnitude, angle)) in enumerate(phasors):
        color_to_use = cycle_colors[i % num_colors]
        x = x_components[i]
        y = y_components[i]
        ax.quiver(current_x, current_y, x, y, angles='xy', scale_units='xy', scale=1, 
                 color=color_to_use, width=input_thickness,
                 label=f'Phasor {i+1}: {magnitude}∠{angle}°')
        current_x += x
        current_y += y
    
    # Plot resultant
    resultant_x = sum(x_components)
    resultant_y = sum(y_components)
    magnitude_resultant = np.sqrt(resultant_x**2 + resultant_y**2)
    angle_resultant = np.degrees(np.arctan2(resultant_y, resultant_x))
    ax.quiver(0, 0, resultant_x, resultant_y, angles='xy', scale_units='xy', scale=1, 
             color='green', width=resultant_thickness,
             label=f'Geometric sum: {magnitude_resultant:.2f}∠{angle_resultant:.1f}°')

    # Set plot parameters
    ax.set_aspect('equal')
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)

    # Set limits
    max_range = max(magnitude_resultant, *[mag for mag, _ in phasors]) * 2
    plt.xlim(-max_range, max_range)
    plt.ylim(-max_range, max_range)

    # Add labels and title
    plt.xlabel('Real')
    plt.ylabel('Imaginary')
    plt.title('Multiple Phasor Addition')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    # Show the plot
    plt.show()

# Example usage:
'''The if __name__ == "__main__": section shows an example usage when running the script directly, 
but you can call the function from anywhere with any valid values. 
The docstring explains what the function does and what parameters it expects.'''

def calculate_alpha_u(Q, p, m, y):
    """
    Calculate alpha_u based on motor parameters.
    
    Parameters:
    Q (int): Number of slots
    p (int): Number of poles
    m (int): Number of phases
    y (int): Winding pitch
    
    Returns:
    float: alpha_u in electrical degrees
    """
    # Calculate basic parameters
    PP = int(p/2)  # Number of pole-pairs
    t = gcd(Q, PP)  # Largest common divider between Q and PP
    
    # Calculate q (slots per pole per phase)
    q = Fraction(Q, p*m)
    z = q.numerator
    n = q.denominator
    
    # Calculate alpha_u
    if t == PP:
        alpha_u = 360 * PP / Q
        alpha_z = alpha_u
    else:
        Qp = Q/t
        alpha_z = 360/Qp
        if n%2 != 0:  # If n is odd => First grade winding
            alpha_u = n * alpha_z
        else:
            alpha_u = n/2 * alpha_z
            
    return alpha_u


import numpy as np
import matplotlib.pyplot as plt
from math import gcd

def plot_phasor_diagram(Q, p, m, alpha_u):
    """
    Generate a phasor diagram and return an array of all phasor numbers in counterclockwise order.
    
    Parameters:
    Q (int): Total number of slots in the motor
    p (int): Total number of poles in the motor
    m (int): Number of phases in the motor
    alpha_u (float): Displacement angle between consecutive phasors in degrees
    
    Returns:
    numpy.ndarray: Array of length Q with phasor numbers in counterclockwise order
    """
    # Derived parameters
    PP = p // 2
    t = gcd(Q, PP)
    Qp = Q / t
    phasors_per_layer = int(Qp)
    
    # Initialize plot
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)
    
    # Layer radii
    radii = [1 + 0.5 * i for i in range(t)]
    
    # Plot parameters
    current_angle = 90  # Start at positive y-axis
    head_length = 0.05
    U_phasors = [1, 8, 7, 2]  # Phase U phasors (red)
    
    # Store phasor positions (number, angle)
    phasor_positions = []
    
    # Plot phasors
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        shaft_scale = 1 - (head_length / radius)
        
        for _ in range(phasors_per_layer):
            angle_rad = np.deg2rad(current_angle)
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            color = 'red' if phasor_number in U_phasors else 'black'
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            text_offset = 1.07 * radius
            text_x = text_offset * np.cos(angle_rad - 3 / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - 3 / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            phasor_positions.append((phasor_number, current_angle))
            phasor_number += 1
            current_angle -= alpha_u  # Clockwise plotting
    
    # Draw layer circles if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)
    
    # Set plot limits and labels
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')
    
    # Save and close
    plt.savefig('phasor_diagram.png')
    plt.close(fig)
    
    # Generate output array
    array_length = Q  # Total phasors across all layers
    # Sort phasors counterclockwise (increasing angle, negate plotted angle)
    phasor_positions.sort(key=lambda x: -x[1])
    phasor_numbers = [p[0] for p in phasor_positions]
    
    # Ensure phasor 1 is first
    idx_1 = phasor_numbers.index(1)
    phasor_numbers = phasor_numbers[idx_1:] + phasor_numbers[:idx_1]
    
    return phasor_positions

if __name__ == "__main__":

    Q = 12 # Q (int): Number of slots
    p = 10 # p (int): Number of poles # "p" is the no. of poles NOTE: Pyrhonen use "p" as Pole-pair
    m = 3 # m (int): Number of phases
    y = 1 # y (int): Winding pitch/ coil throw
    # Define phasors as (magnitude, angle) pairs
    
    # Example 2.23
    # alpha_u calculation requires Q, p, m, and y
    alpha_u = calculate_alpha_u(Q, p, m, y)

    phase_u_array = plot_phasor_diagram(Q, p, m, alpha_u)

    print(phase_u_array)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.show()

# Example usage (can be commented out or removed)
plot_phasor_diagram(Q=12, p=10, m=3)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Initialize list to store phasor number and angle pairs
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number and normalized angle
            phasor_angles.append([phasor_number, normalized_angle])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.close(fig)  # Close figure to free memory

    # Return the array of phasor number and normalized angle pairs
    return phasor_angles

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=12, p=10, m=3)
print(result)

[[1, 90], [2, 300.0], [3, 150.0], [4, 0.0], [5, 210.0], [6, 60.0], [7, 270.0], [8, 120.0], [9, 330.0], [10, 180.0], [11, 30.0], [12, 240.0]]


In [8]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Initialize list to store phasor number and angle pairs
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number and normalized angle
            phasor_angles.append([phasor_number, normalized_angle])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Sort phasor_angles by normalized_angle, starting at 90 degrees
    phasor_angles.sort(key=lambda x: (x[1] - 90) % 360)

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.show()  # Close figure to free memory

    # Return the sorted array of phasor number and normalized angle pairs
    return phasor_angles

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=12, p=10, m=3)
print(result)

[[1, 90], [8, 120.0], [3, 150.0], [10, 180.0], [5, 210.0], [12, 240.0], [7, 270.0], [2, 300.0], [9, 330.0], [4, 0.0], [11, 30.0], [6, 60.0]]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Initialize list to store phasor number and angle pairs
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number and normalized angle
            phasor_angles.append([phasor_number, normalized_angle])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Sort phasor_angles by normalized_angle, starting at 90 degrees
    phasor_angles.sort(key=lambda x: (x[1] - 90) % 360)

    # Assign phase designations
    elements_per_phase = int(Q / t / m / 2)
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    phase_assignments = []
    for phase in phase_order:
        phase_assignments.extend([phase] * elements_per_phase)
    # Extend with "unassigned" for any remaining phasors (if t > 1)
    phase_assignments.extend(["unassigned"] * (Q - len(phase_assignments)))

    # Append phase designations to phasor_angles
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i]] for i, p in enumerate(phasor_angles)]

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.close(fig)  # Close figure to free memory

    # Return the sorted array with phase designations
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=12, p=10, m=3)
print(result)

[[1, 90, 'phase -U'], [13, 90.0, 'phase -U'], [6, 120.0, 'phase +W'], [18, 120.0, 'phase +W'], [11, 150.0, 'phase -V'], [23, 150.0, 'phase -V'], [4, 180.0, 'phase +U'], [16, 180.0, 'phase +U'], [9, 210.0, 'phase -W'], [21, 210.0, 'phase -W'], [2, 240.0, 'phase +V'], [14, 240.0, 'phase +V'], [7, 270.0, 'unassigned'], [19, 270.0, 'unassigned'], [12, 300.0, 'unassigned'], [24, 300.0, 'unassigned'], [5, 330.0, 'unassigned'], [17, 330.0, 'unassigned'], [10, 0.0, 'unassigned'], [22, 0.0, 'unassigned'], [3, 30.0, 'unassigned'], [15, 30.0, 'unassigned'], [8, 60.0, 'unassigned'], [20, 60.0, 'unassigned']]


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Initialize list to store phasor number, angle, and layer
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number, normalized angle, and layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Sort phasor_angles by normalized_angle, starting at 90 degrees
    phasor_angles.sort(key=lambda x: (x[1] - 90) % 360)

    # Assign phase designations
    elements_per_phase = int(Q / t / m / 2)
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    phase_assignments = []
    for phase in phase_order:
        phase_assignments.extend([phase] * elements_per_phase)
    # Extend with "unassigned" for any remaining phasors (if t > 1)
    phase_assignments.extend(["unassigned"] * (Q - len(phase_assignments)))

    # Append phase designations to phasor_angles
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(phasor_angles)]

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.close(fig)  # Close figure to free memory

    # Return the sorted array with phase designations and layer
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=24, p=28, m=3)
print(result)

[[1, 90, 'phase -U', 0], [13, 90.0, 'phase -U', 1], [6, 120.0, 'phase +W', 0], [18, 120.0, 'phase +W', 1], [11, 150.0, 'phase -V', 0], [23, 150.0, 'phase -V', 1], [4, 180.0, 'phase +U', 0], [16, 180.0, 'phase +U', 1], [9, 210.0, 'phase -W', 0], [21, 210.0, 'phase -W', 1], [2, 240.0, 'phase +V', 0], [14, 240.0, 'phase +V', 1], [7, 270.0, 'unassigned', 0], [19, 270.0, 'unassigned', 1], [12, 300.0, 'unassigned', 0], [24, 300.0, 'unassigned', 1], [5, 330.0, 'unassigned', 0], [17, 330.0, 'unassigned', 1], [10, 0.0, 'unassigned', 0], [22, 0.0, 'unassigned', 1], [3, 30.0, 'unassigned', 0], [15, 30.0, 'unassigned', 1], [8, 60.0, 'unassigned', 0], [20, 60.0, 'unassigned', 1]]


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Initialize list to store phasor number, angle, and layer
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number, normalized angle, and layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Sort phasor_angles by normalized_angle (starting at 90 degrees) and then by layer
    phasor_angles.sort(key=lambda x: ((x[1] - 90) % 360, x[2]))

    # Assign phase designations
    elements_per_phase = int(Q / t / m / 2)
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    phase_assignments = []
    for phase in phase_order:
        phase_assignments.extend([phase] * elements_per_phase)
    # Extend with "unassigned" for any remaining phasors (if t > 1)
    phase_assignments.extend(["unassigned"] * (Q - len(phase_assignments)))

    # Append phase designations to phasor_angles
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(phasor_angles)]

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.close(fig)  # Close figure to free memory

    # Return the sorted array with phase designations and layer
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=24, p=4, m=3)
print(result)

[[1, 90, 'phase -U', 0], [13, 90.0, 'phase -U', 1], [12, 120.0, 'phase +W', 0], [24, 120.0, 'phase +W', 1], [11, 150.0, 'phase -V', 0], [23, 150.0, 'phase -V', 1], [10, 180.0, 'phase +U', 0], [22, 180.0, 'phase +U', 1], [9, 210.0, 'phase -W', 0], [21, 210.0, 'phase -W', 1], [8, 240.0, 'phase +V', 0], [20, 240.0, 'phase +V', 1], [7, 270.0, 'unassigned', 0], [19, 270.0, 'unassigned', 1], [6, 300.0, 'unassigned', 0], [18, 300.0, 'unassigned', 1], [5, 330.0, 'unassigned', 0], [17, 330.0, 'unassigned', 1], [4, 0.0, 'unassigned', 0], [16, 0.0, 'unassigned', 1], [3, 30.0, 'unassigned', 0], [15, 30.0, 'unassigned', 1], [2, 60.0, 'unassigned', 0], [14, 60.0, 'unassigned', 1]]


In [20]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = [1, 8, 7, 2]

    # Initialize list to store phasor number, angle, and layer
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number, normalized angle, and layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Group phasors by layer and sort each layer by normalized_angle
    layered_phasors = [[] for _ in range(t)]
    for phasor in phasor_angles:
        layered_phasors[phasor[2]].append(phasor)
    
    # Sort each layer by normalized_angle, starting at 90 degrees
    for layer_phasors in layered_phasors:
        layer_phasors.sort(key=lambda x: (x[1] - 90) % 360)
    
    # Concatenate sorted layers
    sorted_phasors = []
    for layer_phasors in layered_phasors:
        sorted_phasors.extend(layer_phasors)

    # Assign phase designations
    elements_per_phase = int(Q / t / m / 2)
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    phase_assignments = []
    for phase in phase_order:
        phase_assignments.extend([phase] * elements_per_phase)
    # Extend with "unassigned" for any remaining phasors (if t > 1)
    phase_assignments.extend(["unassigned"] * (Q - len(phase_assignments)))

    # Append phase designations to sorted_phasors
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(sorted_phasors)]

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.close(fig)  # Close figure to free memory

    # Return the sorted array with phase designations and layer
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=24, p=28, m=3)
print(result)

[[1, 90, 'phase -U', 0], [6, 120.0, 'phase -U', 0], [11, 150.0, 'phase +W', 0], [4, 180.0, 'phase +W', 0], [9, 210.0, 'phase -V', 0], [2, 240.0, 'phase -V', 0], [7, 270.0, 'phase +U', 0], [12, 300.0, 'phase +U', 0], [5, 330.0, 'phase -W', 0], [10, 0.0, 'phase -W', 0], [3, 30.0, 'phase +V', 0], [8, 60.0, 'phase +V', 0], [13, 90.0, 'unassigned', 1], [18, 120.0, 'unassigned', 1], [23, 150.0, 'unassigned', 1], [16, 180.0, 'unassigned', 1], [21, 210.0, 'unassigned', 1], [14, 240.0, 'unassigned', 1], [19, 270.0, 'unassigned', 1], [24, 300.0, 'unassigned', 1], [17, 330.0, 'unassigned', 1], [22, 0.0, 'unassigned', 1], [15, 30.0, 'unassigned', 1], [20, 60.0, 'unassigned', 1]]


In [23]:
%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    # Derived parameters
    PP = int(p / 2)  # Number of pole-pairs
    t = gcd(Q, PP)   # Largest common divider between Q and PP
    Qp = Q / t       # Total number of phasors per layer
    phasors_per_layer = int(Qp)  # Number of phasors in a single layer

    # Calculate slot per pole per phase
    q = Fraction(Q, p * m)
    z = q.numerator
    n = q.denominator

    # Calculate alpha_u and alpha_z
    if t == PP:
        alpha_u = 360 * PP / Q  # Normal calculation if t == PP
        alpha_z = alpha_u
    else:
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first grade winding)
            alpha_u = n * alpha_z
        else:
            alpha_u = n / 2 * alpha_z

    # Initialize the figure and axis for plotting
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure circular appearance
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # Dynamically generate the radius for each layer
    radii = [1 + 0.5 * i for i in range(t)]

    # Starting angle for the first phasor (90 degrees = positive y-axis)
    current_angle = 90

    # Define arrowhead length
    head_length = 0.05  # Length of the arrowhead in plot units

    # Define phasors to be colored red (example values)
    U_phasors = []

    # Initialize list to store phasor number, angle, and layer
    phasor_angles = []

    # Plot phasors for each layer
    phasor_number = 1
    for layer in range(t):
        radius = radii[layer]
        # Calculate shaft scale to ensure arrow tip touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize current_angle to [0, 360)
            normalized_angle = current_angle % 360
            # Store phasor number, normalized angle, and layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # Convert angle to radians
            angle_rad = np.deg2rad(current_angle)
            
            # Calculate coordinates of phasor tip
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            
            # Determine color
            color = 'red' if phasor_number in U_phasors else 'black'
            
            # Plot phasor as an arrow
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # Add phasor number near arrow tip
            text_offset = 1.07 * radius
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number
            phasor_number += 1
            
            # Update angle for next phasor (clockwise)
            current_angle -= alpha_u

    # Group phasors by layer and sort each layer by normalized_angle
    layered_phasors = [[] for _ in range(t)]
    for phasor in phasor_angles:
        layered_phasors[phasor[2]].append(phasor)
    
    # Sort each layer by normalized_angle, starting at 90 degrees
    for layer_phasors in layered_phasors:
        layer_phasors.sort(key=lambda x: (x[1] - 90) % 360)
    
    # Concatenate sorted layers
    sorted_phasors = []
    for layer_phasors in layered_phasors:
        sorted_phasors.extend(layer_phasors)

    # Assign phase designations
    elements_per_phase = int(Q / t / m / 2)
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    phase_assignments = []
    # Repeat phase_order to cover all Q phasors
    total_phasors = Q
    while len(phase_assignments) < total_phasors:
        for phase in phase_order:
            phase_assignments.extend([phase] * elements_per_phase)
            if len(phase_assignments) >= total_phasors:
                break
    # Truncate to exactly Q elements
    phase_assignments = phase_assignments[:total_phasors]

    # Append phase designations to sorted_phasors
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(sorted_phasors)]

    # Draw circles for each layer if t > 1
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # Set plot limits
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # Add labels and title
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title('Phasor Diagram')

    # Save the plot
    plt.savefig('phasor_diagram.png')
    plt.show()

    # Return the sorted array with phase designations and layer
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=24, p=28, m=3)
print(result)

[[1, 90, 'phase -U', 0], [6, 120.0, 'phase -U', 0], [11, 150.0, 'phase +W', 0], [4, 180.0, 'phase +W', 0], [9, 210.0, 'phase -V', 0], [2, 240.0, 'phase -V', 0], [7, 270.0, 'phase +U', 0], [12, 300.0, 'phase +U', 0], [5, 330.0, 'phase -W', 0], [10, 0.0, 'phase -W', 0], [3, 30.0, 'phase +V', 0], [8, 60.0, 'phase +V', 0], [13, 90.0, 'phase -U', 1], [18, 120.0, 'phase -U', 1], [23, 150.0, 'phase +W', 1], [16, 180.0, 'phase +W', 1], [21, 210.0, 'phase -V', 1], [14, 240.0, 'phase -V', 1], [19, 270.0, 'phase +U', 1], [24, 300.0, 'phase +U', 1], [17, 330.0, 'phase -W', 1], [22, 0.0, 'phase -W', 1], [15, 30.0, 'phase +V', 1], [20, 60.0, 'phase +V', 1]]


In [25]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    """
    Generates a phasor diagram for an electric motor and returns an array of phasor details.
    
    Parameters:
    - Q: Total number of slots in the motor.
    - p: Total number of poles in the motor.
    - m: Number of phases in the motor (e.g., 3 for a three-phase motor).
    
    Returns:
    - A list of [phasor_number, normalized_angle, phase, layer] quadruples, where:
      - phasor_number: Integer from 1 to Q, identifying each phasor.
      - normalized_angle: Angle in degrees [0, 360), representing the phasor's position.
      - phase: Phase designation ("phase -U", "phase +W", etc.).
      - layer: Layer index (0 to t-1), where t is the number of layers.
    The array is sorted first by layer (0, 1, ..., t-1), then by normalized_angle within each layer.
    """
    # --- Calculate derived parameters ---
    # PP: Number of pole-pairs (half the number of poles)
    PP = int(p / 2)
    # t: Greatest common divisor between Q and PP, determines the number of layers
    t = gcd(Q, PP)
    # Qp: Number of phasors per layer (Q divided by t)
    Qp = Q / t
    # phasors_per_layer: Integer number of phasors in each layer
    phasors_per_layer = int(Qp)

    # --- Calculate slot per pole per phase ---
    # q: Fraction representing slots per pole per phase (Q / (p * m))
    q = Fraction(Q, p * m)
    z = q.numerator  # Numerator of the fraction
    n = q.denominator  # Denominator of the fraction, used to determine winding type

    # --- Calculate angular displacements ---
    # alpha_u: Angular displacement between consecutive phasors in degrees
    # alpha_z: Base angle for phasor spacing
    if t == PP:
        # Case 1: When t equals PP, use standard angle calculation
        alpha_u = 360 * PP / Q
        alpha_z = alpha_u
    else:
        # Case 2: For fractional slot designs, adjust angles based on Pyrhonen's method
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first-grade winding)
            alpha_u = n * alpha_z
        else:  # If n is even
            alpha_u = n / 2 * alpha_z

    # --- Set up the plot ---
    # Create a figure and axis for the phasor diagram (5x5 inches, 150 DPI)
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure the plot is circular (equal scaling on axes)
    # Add a light gray grid for better visualization
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # --- Define layer radii ---
    # Each layer has a radius starting at 1, increasing by 0.5 per layer
    # For t=1, radii = [1]; for t=2, radii = [1, 1.5], etc.
    radii = [1 + 0.5 * i for i in range(t)]

    # --- Initialize plotting parameters ---
    # Start the first phasor at 90 degrees (positive y-axis, top of the plot)
    current_angle = 90
    # Define the length of the arrowhead (in plot units, kept small)
    head_length = 0.05
    # List of phasor numbers to be colored red in the plot (example values)
    U_phasors = []

    # --- Initialize array to store phasor data ---
    # Will store [phasor_number, normalized_angle, layer] temporarily
    phasor_angles = []

    # --- Plot phasors and collect data ---
    phasor_number = 1  # Start numbering phasors from 1
    for layer in range(t):
        # Get the radius for the current layer (e.g., 1 for layer 0, 1.5 for layer 1)
        radius = radii[layer]
        # Adjust shaft length so the arrow tip (including head) touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize the current angle to [0, 360) to avoid negative/large angles
            normalized_angle = current_angle % 360
            # Store phasor data: number, angle, and layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # --- Plot the phasor ---
            # Convert angle to radians for trigonometric calculations
            angle_rad = np.deg2rad(current_angle)
            # Calculate x, y coordinates of the arrow tip, scaled to account for arrowhead
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            # Color the phasor red if its number is in U_phasors, else black
            color = 'red' if phasor_number in U_phasors else 'black'
            # Draw an arrow from origin (0,0) to (x,y) with specified head size
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # --- Add phasor number label ---
            # Place text slightly outside the circle for readability
            text_offset = 1.07 * radius
            # Slight angular offset for text positioning
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            # Add the phasor number as text, centered at (text_x, text_y)
            ax.text(text_x, text_y, str(phasor_number), fontsize=10, ha='center', va='center')
            
            # Increment phasor number for the next phasor
            phasor_number += 1
            # Move to the next phasor angle (clockwise, subtract alpha_u)
            current_angle -= alpha_u

    # --- Sort phasors by layer and angle ---
    # Create a list of t empty lists to group phasors by layer
    layered_phasors = [[] for _ in range(t)]
    # Assign each phasor to its layer's list
    for phasor in phasor_angles:
        layered_phasors[phasor[2]].append(phasor)
    
    # Sort each layer's phasors by normalized_angle, starting at 90 degrees
    # The key (x[1] - 90) % 360 transforms angles so 90° maps to 0, 91° to 1, ..., 89° to 359
    for layer_phasors in layered_phasors:
        layer_phasors.sort(key=lambda x: (x[1] - 90) % 360)
    
    # Concatenate sorted layers into a single list
    # This ensures layer 0 phasors come first, then layer 1, etc.
    sorted_phasors = []
    for layer_phasors in layered_phasors:
        sorted_phasors.extend(layer_phasors)

    # --- Assign phase designations ---
    # Each phase group has Q/t/m/2 phasors
    elements_per_phase = int(Q / t / m / 2)
    # Define the phase order to cycle through
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    phase_assignments = []
    # Repeat phase_order to cover all Q phasors
    total_phasors = Q
    while len(phase_assignments) < total_phasors:
        for phase in phase_order:
            # Add elements_per_phase copies of the current phase
            phase_assignments.extend([phase] * elements_per_phase)
            if len(phase_assignments) >= total_phasors:
                break
    # Ensure exactly Q assignments
    phase_assignments = phase_assignments[:total_phasors]

    # --- Create final array with phase designations ---
    # Combine sorted phasors with their phase assignments
    # Format: [phasor_number, normalized_angle, phase, layer]
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(sorted_phasors)]

    # --- Draw layer circles (if t > 1) ---
    # Add circles to visualize layers, only if there are multiple layers
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # --- Set plot limits ---
    # Ensure all phasors and circles are visible by setting limits based on the largest radius
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # --- Add plot labels and title ---
    ax.set_xlabel('Real')  # X-axis represents the real part of the phasor
    ax.set_ylabel('Imaginary')  # Y-axis represents the imaginary part
    ax.set_title('Phasor Diagram')  # Title of the plot

    # --- Save and close the plot ---
    plt.savefig('phasor_diagram.png')  # Save the plot as an image
    plt.show()
    # plt.close(fig)  # Close the figure to free memory

    # --- Return the final array ---
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=12, p=10, m=3)
print(result)

[[1, 90, 'phase -U', 0], [8, 120.0, 'phase -U', 0], [3, 150.0, 'phase +W', 0], [10, 180.0, 'phase +W', 0], [5, 210.0, 'phase -V', 0], [12, 240.0, 'phase -V', 0], [7, 270.0, 'phase +U', 0], [2, 300.0, 'phase +U', 0], [9, 330.0, 'phase -W', 0], [4, 0.0, 'phase -W', 0], [11, 30.0, 'phase +V', 0], [6, 60.0, 'phase +V', 0]]


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    """
    Generates a phasor diagram for an electric motor and returns an array of phasor details.
    
    Parameters:
    - Q: Total number of slots in the motor.
    - p: Total number of poles in the motor.
    - m: Number of phases in the motor (e.g., 3 for a three-phase motor).
    
    Returns:
    - A list of [phasor_number, normalized_angle, phase, layer] quadruples, where:
      - phasor_number: Integer from 1 to Q, identifying each phasor.
      - normalized_angle: Angle in degrees [0, 360), representing the phasor's position.
      - phase: Phase designation ("phase -U", "phase +W", etc.).
      - layer: Layer index (0 to t-1), where t is the number of layers.
    The array is sorted first by layer (0, 1, ..., t-1), then by normalized_angle within each layer.
    The plot includes phasor numbers and shortened phase designations (e.g., "-U") with reduced text size.
    """
    # --- Calculate derived parameters ---
    # PP: Number of pole-pairs (half the number of poles)
    PP = int(p / 2)
    # t: Greatest common divisor between Q and PP, determines the number of layers
    t = gcd(Q, PP)
    # Qp: Number of phasors per layer (Q divided by t)
    Qp = Q / t
    # phasors_per_layer: Integer number of phasors in each layer
    phasors_per_layer = int(Qp)

    # --- Calculate slot per pole per phase ---
    # q: Fraction representing slots per pole per phase (Q / (p * m))
    q = Fraction(Q, p * m)
    z = q.numerator  # Numerator of the fraction
    n = q.denominator  # Denominator of the fraction, used to determine winding type

    # --- Calculate angular displacements ---
    # alpha_u: Angular displacement between consecutive phasors in degrees
    # alpha_z: Base angle for phasor spacing
    if t == PP:
        # Case 1: When t equals PP, use standard angle calculation
        alpha_u = 360 * PP / Q
        alpha_z = alpha_u
    else:
        # Case 2: For fractional slot designs, adjust angles based on Pyrhonen's method
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first-grade winding)
            alpha_u = n * alpha_z
        else:  # If n is even
            alpha_u = n / 2 * alpha_z

    # --- Set up the plot ---
    # Create a figure and axis for the phasor diagram (5x5 inches, 150 DPI)
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure the plot is circular (equal scaling on axes)
    # Add a light gray grid for better visualization
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # --- Define layer radii ---
    # Each layer has a radius starting at 1, increasing by 0.5 per layer
    # For t=1, radii = [1]; for t=2, radii = [1, 1.5], etc.
    radii = [1 + 0.5 * i for i in range(t)]

    # --- Initialize plotting parameters ---
    # Start the first phasor at 90 degrees (positive y-axis, top of the plot)
    current_angle = 90
    # Define the length of the arrowhead (in plot units, kept small)
    head_length = 0.05
    # List of phasor numbers to be colored red in the plot (example values)
    U_phasors = [1, 8, 7, 2]

    # --- Initialize arrays to store phasor data and phases ---
    # phasor_angles: Stores [phasor_number, normalized_angle, layer, phase] temporarily
    phasor_angles = []
    # phase_assignments_plot: Stores phase designations for plotting order
    phase_assignments_plot = []

    # --- Compute phase assignments for plotting ---
    # Each phase group has Q/t/m/2 phasors
    elements_per_phase = int(Q / t / m / 2)
    # Define the phase order to cycle through
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    # Map full phase names to short forms for plot labels
    phase_short_map = {
        "phase -U": "-U", "phase +W": "+W", "phase -V": "-V",
        "phase +U": "+U", "phase -W": "-W", "phase +V": "+V"
    }
    # Generate phase assignments for all Q phasors
    total_phasors = Q
    while len(phase_assignments_plot) < total_phasors:
        for phase in phase_order:
            phase_assignments_plot.extend([phase] * elements_per_phase)
            if len(phase_assignments_plot) >= total_phasors:
                break
    phase_assignments_plot = phase_assignments_plot[:total_phasors]

    # --- Plot phasors and collect data ---
    phasor_number = 1  # Start numbering phasors from 1
    phase_idx = 0  # Index for phase assignments in plotting order
    for layer in range(t):
        # Get the radius for the current layer (e.g., 1 for layer 0, 1.5 for layer 1)
        radius = radii[layer]
        # Adjust shaft length so the arrow tip (including head) touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize the current angle to [0, 360) to avoid negative/large angles
            normalized_angle = current_angle % 360
            # Get the phase for this phasor in plotting order
            current_phase = phase_assignments_plot[phase_idx]
            # Store phasor data: number, angle, layer, and phase
            phasor_angles.append([phasor_number, normalized_angle, layer, current_phase])
            
            # --- Plot the phasor ---
            # Convert angle to radians for trigonometric calculations
            angle_rad = np.deg2rad(current_angle)
            # Calculate x, y coordinates of the arrow tip, scaled to account for arrowhead
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            # Color the phasor red if its number is in U_phasors, else black
            color = 'red' if phasor_number in U_phasors else 'black'
            # Draw an arrow from origin (0,0) to (x,y) with specified head size
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # --- Add phasor number and phase label ---
            # Place text slightly outside the circle for readability
            text_offset = 1.07 * radius
            # Slight angular offset for text positioning
            text_angle_offset = 3
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            # Combine phasor number and shortened phase (e.g., "1 (-U)")
            label = f"{phasor_number} ({phase_short_map[current_phase]})"
            # Add the text with reduced font size
            ax.text(text_x, text_y, label, fontsize=8, ha='center', va='center')
            
            # Increment phasor number and phase index
            phasor_number += 1
            phase_idx += 1
            # Move to the next phasor angle (clockwise, subtract alpha_u)
            current_angle -= alpha_u

    # --- Sort phasors by layer and angle ---
    # Create a list of t empty lists to group phasors by layer
    layered_phasors = [[] for _ in range(t)]
    # Assign each phasor to its layer's list (omit phase for now)
    for phasor in phasor_angles:
        layered_phasors[phasor[2]].append([phasor[0], phasor[1], phasor[2]])
    
    # Sort each layer's phasors by normalized_angle, starting at 90 degrees
    # The key (x[1] - 90) % 360 transforms angles so 90° maps to 0, 91° to 1, ..., 89° to 359
    for layer_phasors in layered_phasors:
        layer_phasors.sort(key=lambda x: (x[1] - 90) % 360)
    
    # Concatenate sorted layers into a single list
    # This ensures layer 0 phasors come first, then layer 1, etc.
    sorted_phasors = []
    for layer_phasors in layered_phasors:
        sorted_phasors.extend(layer_phasors)

    # --- Assign phase designations for output array ---
    # Recompute phases for the sorted array (may differ from plotting order)
    phase_assignments = []
    while len(phase_assignments) < total_phasors:
        for phase in phase_order:
            phase_assignments.extend([phase] * elements_per_phase)
            if len(phase_assignments) >= total_phasors:
                break
    phase_assignments = phase_assignments[:total_phasors]

    # --- Create final array with phase designations ---
    # Combine sorted phasors with their phase assignments
    # Format: [phasor_number, normalized_angle, phase, layer]
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(sorted_phasors)]

    # --- Draw layer circles (if t > 1) ---
    # Add circles to visualize layers, only if there are multiple layers
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # --- Set plot limits ---
    # Ensure all phasors and circles are visible by setting limits based on the largest radius
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # --- Add plot labels and title ---
    ax.set_xlabel('Real')  # X-axis represents the real part of the phasor
    ax.set_ylabel('Imaginary')  # Y-axis represents the imaginary part
    ax.set_title('Phasor Diagram')  # Title of the plot

    # --- Save and close the plot ---
    plt.savefig('phasor_diagram.png')  # Save the plot as an image
    plt.show()
    # plt.close(fig)  # Close the figure to free memory

    # --- Return the final array ---
    return phasor_angles_with_phase

# Example usage (can be commented out or removed)
result = plot_phasor_diagram(Q=12, p=10, m=3)
print(result)

[[1, 90, 'phase -U', 0], [8, 120.0, 'phase -U', 0], [3, 150.0, 'phase +W', 0], [10, 180.0, 'phase +W', 0], [5, 210.0, 'phase -V', 0], [12, 240.0, 'phase -V', 0], [7, 270.0, 'phase +U', 0], [2, 300.0, 'phase +U', 0], [9, 330.0, 'phase -W', 0], [4, 0.0, 'phase -W', 0], [11, 30.0, 'phase +V', 0], [6, 60.0, 'phase +V', 0]]


In [39]:
import numpy as np
import matplotlib.pyplot as plt
from math import gcd
from fractions import Fraction

def plot_phasor_diagram(Q, p, m):
    """
    Generates a phasor diagram for an electric motor and returns an array of phasor details.
    
    Parameters:
    - Q: Total number of slots in the motor.
    - p: Total number of poles in the motor.
    - m: Number of phases in the motor (e.g., 3 for a three-phase motor).
    
    Returns:
    - A list of [phasor_number, normalized_angle, phase, layer] quadruples, where:
      - phasor_number: Integer from 1 to Q, identifying each phasor.
      - normalized_angle: Angle in degrees [0, 360), representing the phasor's position.
      - phase: Phase designation ("phase -U", "phase +W", etc.).
      - layer: Layer index (0 to t-1), where t is the number of layers.
    The array is sorted first by layer (0, 1, ..., t-1), then by normalized_angle within each layer.
    The plot includes phasor numbers and phase designations (e.g., "-U") matching the sorted array, with reduced text size.
    """
    # --- Calculate derived parameters ---
    # PP: Number of pole-pairs (half the number of poles)
    PP = int(p / 2)
    # t: Greatest common divisor between Q and PP, determines the number of layers
    t = gcd(Q, PP)
    # Qp: Number of phasors per layer (Q divided by t)
    Qp = Q / t
    # phasors_per_layer: Integer number of phasors in each layer
    phasors_per_layer = int(Qp)

    # --- Calculate slot per pole per phase ---
    # q: Fraction representing slots per pole per phase (Q / (p * m))
    q = Fraction(Q, p * m)
    z = q.numerator  # Numerator of the fraction
    n = q.denominator  # Denominator of the fraction, used to determine winding type

    # --- Calculate angular displacements ---
    # alpha_u: Angular displacement between consecutive phasors in degrees
    # alpha_z: Base angle for phasor spacing
    if t == PP:
        # Case 1: When t equals PP, use standard angle calculation
        alpha_u = 360 * PP / Q
        alpha_z = alpha_u
    else:
        # Case 2: For fractional slot designs, adjust angles based on Pyrhonen's method
        alpha_z = 360 / Qp
        if n % 2 != 0:  # If n is odd (first-grade winding)
            alpha_u = n * alpha_z
        else:  # If n is even
            alpha_u = n / 2 * alpha_z

    # --- Set up the plot ---
    # Create a figure and axis for the phasor diagram (5x5 inches, 150 DPI)
    fig, ax = plt.subplots(figsize=(5, 5), dpi=150)
    ax.set_aspect('equal')  # Ensure the plot is circular (equal scaling on axes)
    # Add a light gray grid for better visualization
    ax.grid(True, color='lightgray', linestyle='--', linewidth=0.5, alpha=0.5)

    # --- Define layer radii ---
    # Each layer has a radius starting at 1, increasing by 0.5 per layer
    # For t=1, radii = [1]; for t=2, radii = [1, 1.5], etc.
    radii = [1 + 0.5 * i for i in range(t)]

    # --- Initialize plotting parameters ---
    # Start the first phasor at 90 degrees (positive y-axis, top of the plot)
    current_angle = 90
    # Define the length of the arrowhead (in plot units, kept small)
    head_length = 0.05
    # List of phasor numbers to be colored red in the plot (example values)
    U_phasors = []

    # --- Initialize array to store phasor data ---
    # phasor_angles: Stores [phasor_number, normalized_angle, layer] temporarily
    phasor_angles = []

    # --- Collect phasor data during plotting ---
    phasor_number = 1  # Start numbering phasors from 1
    for layer in range(t):
        # Get the radius for the current layer (e.g., 1 for layer 0, 1.5 for layer 1)
        radius = radii[layer]
        # Adjust shaft length so the arrow tip (including head) touches the circle
        shaft_scale = 1 - (head_length / radius)
        
        for i in range(phasors_per_layer):
            # Normalize the current angle to [0, 360) to avoid negative/large angles
            normalized_angle = current_angle % 360
            # Store phasor data: number, angle, layer
            phasor_angles.append([phasor_number, normalized_angle, layer])
            
            # --- Plot the phasor ---
            # Convert angle to radians for trigonometric calculations
            angle_rad = np.deg2rad(current_angle)
            # Calculate x, y coordinates of the arrow tip, scaled to account for arrowhead
            x = radius * shaft_scale * np.cos(angle_rad)
            y = radius * shaft_scale * np.sin(angle_rad)
            # Color the phasor red if its number is in U_phasors, else black
            color = 'red' if phasor_number in U_phasors else 'black'
            # Draw an arrow from origin (0,0) to (x,y) with specified head size
            ax.arrow(0, 0, x, y, head_width=0.04, head_length=0.05, fc=color, ec=color)
            
            # --- Placeholder for text label (added after phase assignment) ---
            # Store text position for later use
            text_offset = 1.08 * radius
            text_angle_offset = 7
            text_x = text_offset * np.cos(angle_rad - text_angle_offset / 180 * np.pi)
            text_y = text_offset * np.sin(angle_rad - text_angle_offset / 180 * np.pi)
            phasor_angles[-1].append((text_x, text_y))  # Append text coordinates
            
            # Increment phasor number
            phasor_number += 1
            # Move to the next phasor angle (clockwise, subtract alpha_u)
            current_angle -= alpha_u

    # --- Sort phasors by layer and angle ---
    # Create a list of t empty lists to group phasors by layer
    layered_phasors = [[] for _ in range(t)]
    # Assign each phasor to its layer's list
    for phasor in phasor_angles:
        layered_phasors[phasor[2]].append([phasor[0], phasor[1], phasor[2], phasor[3]])
    
    # Sort each layer's phasors by normalized_angle, starting at 90 degrees
    # The key (x[1] - 90) % 360 transforms angles so 90° maps to 0, 91° to 1, ..., 89° to 359
    for layer_phasors in layered_phasors:
        layer_phasors.sort(key=lambda x: (x[1] - 90) % 360)
    
    # Concatenate sorted layers into a single list
    # This ensures layer 0 phasors come first, then layer 1, etc.
    sorted_phasors = []
    for layer_phasors in layered_phasors:
        sorted_phasors.extend(layer_phasors)

    # --- Assign phase designations ---
    # Each phase group has Q/t/m/2 phasors
    elements_per_phase = int(Q / t / m / 2)
    # Define the phase order to cycle through
    phase_order = ["phase -U", "phase +W", "phase -V", "phase +U", "phase -W", "phase +V"]
    # Generate phase assignments for all Q phasors
    total_phasors = Q
    phase_assignments = []
    while len(phase_assignments) < total_phasors:
        for phase in phase_order:
            phase_assignments.extend([phase] * elements_per_phase)
            if len(phase_assignments) >= total_phasors:
                break
    phase_assignments = phase_assignments[:total_phasors]

    # --- Create mapping from phasor_number to phase ---
    # Map phasor numbers to their phases in the sorted array
    phasor_to_phase = {p[0]: phase_assignments[i] for i, p in enumerate(sorted_phasors)}

    # --- Map full phase names to short forms for plot labels ---
    phase_short_map = {
        "phase -U": "-U", "phase +W": "+W", "phase -V": "-V",
        "phase +U": "+U", "phase -W": "-W", "phase +V": "+V"
    }

    # --- Add text labels to the plot ---
    # Iterate through phasor_angles to add labels using sorted phases
    for phasor in phasor_angles:
        phasor_number = phasor[0]
        text_x, text_y = phasor[3]  # Text coordinates stored during plotting
        # Get the phase from the sorted array's mapping
        phase = phasor_to_phase[phasor_number]
        # Combine phasor number and shortened phase (e.g., "1 (-U)")
        label = f"{phasor_number} ({phase_short_map[phase]})"
        # Add the text with reduced font size
        ax.text(text_x, text_y, label, fontsize=6, ha='center', va='center')

    # --- Create final array with phase designations ---
    # Format: [phasor_number, normalized_angle, phase, layer]
    phasor_angles_with_phase = [[p[0], p[1], phase_assignments[i], p[2]] for i, p in enumerate(sorted_phasors)]

    # --- Draw layer circles (if t > 1) ---
    # Add circles to visualize layers, only if there are multiple layers
    if t > 1:
        for radius in radii:
            circle = plt.Circle((0, 0), radius, fill=False, color='gray')
            ax.add_patch(circle)

    # --- Set plot limits ---
    # Ensure all phasors and circles are visible by setting limits based on the largest radius
    max_radius = max(radii)
    ax.set_xlim(-max_radius - 0.5, max_radius + 0.5)
    ax.set_ylim(-max_radius - 0.5, max_radius + 0.5)

    # --- Add plot labels and title ---
    ax.set_xlabel('Real')  # X-axis represents the real part of the phasor
    ax.set_ylabel('Imaginary')  # Y-axis represents the imaginary part
    ax.set_title('Phasor Diagram')  # Title of the plot

    # --- Save and close the plot ---
    plt.savefig('phasor_diagram.png')  # Save the plot as an image
    plt.show()
    # plt.close(fig)  # Close the figure to free memory

    # --- Return the final array ---
    return phasor_angles_with_phase

# Example usage
result = plot_phasor_diagram(Q=12, p=10, m=3)
print(result)

[[1, 90, 'phase -U', 0], [8, 120.0, 'phase -U', 0], [3, 150.0, 'phase +W', 0], [10, 180.0, 'phase +W', 0], [5, 210.0, 'phase -V', 0], [12, 240.0, 'phase -V', 0], [7, 270.0, 'phase +U', 0], [2, 300.0, 'phase +U', 0], [9, 330.0, 'phase -W', 0], [4, 0.0, 'phase -W', 0], [11, 30.0, 'phase +V', 0], [6, 60.0, 'phase +V', 0]]
